In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from scipy.optimize import minimize

In [2]:
from dask.distributed import Client
client = Client()

In [3]:
vcf_dir = "/home/djl34/scratch/genetics.bwh.harvard.edu/downloads/Vova/Roulette"

In [4]:
df = dd.read_csv(vcf_dir + "/all_hq_synonymous_variants.tsv.gz", sep = "\t")

/home/djl34/.conda/envs/kl/lib/python3.10/site-packages/dask/dataframe/io/csv.py:531: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [5]:
df_observed = dd.read_csv("polymorphic_variants.tsv", sep = "\t")

In [6]:
df_observed["polymorphic"] = 1

In [7]:
df_merged = df[["CHROM", "POS", "REF", "ALT", "mu"]].merge(df_observed[["CHROM", "POS", "REF", "ALT", "polymorphic"]], how = "left", on = ["CHROM", "POS", "REF", "ALT"])

In [8]:
df_group = pd.DataFrame(df.groupby("mu").agg({"polymorphic": ['size', 'sum']}).compute())

In [9]:
df_group.columns = df_group.columns.droplevel()

In [10]:
df_group = df_group.rename({'size': "total", "sum": "polymorphic"}, axis = 1)
df_group = df_group.reset_index()

In [11]:
df_group

,mu,total,polymorphic
0,0.400,10419,3750
1,0.186,243144,54654
2,0.139,708827,126478
3,0.342,21608,6862
4,0.073,1132718,113316
...,...,...,...
94,0.799,1397,740
95,3.219,2398,2275
96,3.507,525,505
97,3.912,97,90


In [12]:
df_group["poisson_lambda"] = 1 - np.exp(-1*df_group["mu"])

df_group["expected_polymorphic"] = df_group["poisson_lambda"] * df_group["total"]

def poisson_function(x):    
    return abs(sum((1 - np.exp(-1 * x * df_group["mu"]))*df_group["total"]) - sum(df_group["polymorphic"]))

x0 = np.array([1])
res = minimize(poisson_function, x0, method='Nelder-Mead', options={'xatol': 1e-3, 'disp': True})

# this is the proper scaling factor for the mutation rate
k = res.x[0]

Optimization terminated successfully.
         Current function value: 0.000239
         Iterations: 32
         Function evaluations: 64


In [13]:
k

1.3767815530300145

In [8]:
df[df["polymorphic"] == 1].to_csv("polymorphic_variants.tsv", sep = "\t", index = None)